# Informed Search Algorithms #


In the last session, we've implemented different systematic search strategies. If we want to find paths between different cities in a map, we can use additional information to guide our search (a heuristic). We don't rely on the 'blind' search and can implement more efficient algorithms, that consider the coordinates of the SBB hubs and use the aerial distances between them.

Implement the following algorithms and answer the questions on ILIAS for the **testat** exercise.

1. Greedy Search
1. A* Algorithm
1. IDA* Search (optional, not needed for testat exercise)

Hints: 
- The aerial distance between a node and the goal can be computed with the following function:

    `sbb.get_distance_between(node.state, problem.goal)`
    

-  You will use a prioritq queue for the frontier. You can use the heap library `heapq` for example:

    `from heapq import heappush, heappop`

    The following line will add the node `node` to the frontier with priority `f`:

    `heappush(frontier, (f, node))`

    To get the first node (the one with the lowest value of f) use: `node = heappop(frontier)[1]`.
    
    An example of such a queue is given below.
    

- Keep track of the number of nodes that are stored simultaneously.



In [11]:
!git clone https://github.com/iaherzog/search.git

class Graph:

    """A graph connects nodes by edges.  Each edge can also
    have a length associated with it.  The constructor call is something like:
        g = Graph({'A': {'B': 1, 'C': 2})
    this makes a graph with 3 nodes, A, B, and C, with an edge of length 1 from
    A to B,  and an edge of length 2 from A to C.  You can also do:
        g = Graph({'A': {'B': 1, 'C': 2}, directed=False)
    This makes an undirected graph, so inverse links are also added. The graph
    stays undirected; if you add more edges with g.connect('B', 'C', 3), then
    inverse link is also added.  You can use g.get_nodes() to get a list of nodes,
    g.get_edges('A') to get a dict of edges out of A, and g.get_distance('A', 'B') to get the
    length of the edge from A to B. """

    def __init__(self, graph_dict=None, directed=True):
        self.graph_dict = graph_dict or {}
        self.directed = directed
        if not directed:
            self.make_undirected()

    def make_undirected(self):
        """Make a digraph into an undirected graph by adding symmetric edges."""
        for a in list(self.graph_dict.keys()):
            for (b, dist) in self.graph_dict[a].items():
                self.add_connection(b, a, dist)

    def connect(self, A, B, distance=1):
        """Add a link from A and B of given distance, and also add the inverse
        link if the graph is undirected."""
        self.add_connection(A, B, distance)
        if not self.directed:
            self.add_connection(B, A, distance)

    def add_connection(self, A, B, distance):
        """Add a link from A to B of given distance, in one direction only."""
        self.graph_dict.setdefault(A, {})[B] = distance

    def get_edges(self, a):
        return self.graph_dict.setdefault(a, {})

    def get_distance(self, a, b):
        links = self.graph_dict.setdefault(a, {})
        return links.get(b)

    def get_nodes(self):
        """Return a list of nodes in the graph."""
        s1 = set([k for k in self.graph_dict.keys()])
        s2 = set([k2 for v in self.graph_dict.values() for k2, v2 in v.items()])
        nodes = s1.union(s2)
        return list(nodes)


def UndirectedGraph(graph_dict=None):
    """Build a Graph where every edge (including future ones) goes both ways."""
    return Graph(graph_dict = graph_dict, directed=False)


class GraphProblem():

    """The problem of searching a graph from one node to another."""

    def __init__(self, initial, goal, graph):
        self.initial = initial
        self.goal = goal
        self.graph = graph

    def goal_test(self, state):
        """Return True if the state is the goal. """
        return state == self.goal

    def get_actions_from(self, A):
        """The actions at a graph node are just its neighbors."""
        return list(self.graph.get_edges(A).keys())

    def get_path_cost(self, A, B):
        return self.graph.get_distance(A, B)



class Node:
    """A node in a search tree."""

    def __init__(self, state, parent=None, action=None, path_cost=0):
        """Create a search tree Node, derived from a parent by an action."""
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def expand(self, problem):
        """List the nodes reachable in one step from this node."""
        actions = problem.get_actions_from(self.state)
        successors = []
        for action in actions:
            successors.append(self.create_child_node(problem, action))
        return successors

    def create_child_node(self, problem, action):
        next_state = action
        next_node = Node(next_state, parent=self, action=action,
                         path_cost=self.path_cost + problem.get_path_cost(self.state, next_state))
        return next_node

    def get_path_from_root(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

    def get_solution(self):
        """Return the sequence of actions to go from the root to this node."""
        return [node.action for node in self.get_path_from_root()[1:]]


Cloning into 'search'...


In [12]:
import sys
sys.path.append('content/search')

from heapq import heappush, heappop
priority_queue = []

heappush(priority_queue, (10, 'node_with_priority_10'))
heappush(priority_queue, (20, 'node_with_priority_20'))
heappush(priority_queue, (5, 'node_with_priority_5'))

# get the element with the lowest value (note, the following function returns both the priority and the element)
print(heappop(priority_queue))

(5, 'node_with_priority_5')


In [13]:
# implement the informed search algorithms here
# problem is an instance of the GraphProblem class defined in search.py
# and heuristic is a function that takes two arguments (node.state, goal_node.state) and calculates h(node)

def greedy_search(problem, heuristic):
    node = Node(problem.initial)

    frontier = []
    explored = set()
    highest_amt_stored_nodes = 1;

    heappush(frontier, (heuristic(problem.goal, node.state), node))

    while len(frontier) > 0:
      node = heappop(frontier)[1]

      if highest_amt_stored_nodes < len(frontier):
            highest_amt_stored_nodes = len(frontier)

      if problem.goal_test(node.state):
        print("Nodes still left: ", len(frontier), "explored: ", len(explored), "most simultaneously stored nodes: ", highest_amt_stored_nodes)
        return node
      
      explored.add(node.state)
      for childNode in node.expand(problem):
            if childNode.state not in explored and not childNode.state in (x[1].state for x in frontier):
                distance = heuristic(problem.goal, childNode.state)
                heappush(frontier, (distance, childNode))         
    return None

def a_star_search(problem, heuristic):
    node = Node(problem.initial)

    frontier = []
    explored = set()
    highest_amt_stored_nodes = 1;
    
    f_n = heuristic(problem.goal, node.state)
    heappush(frontier, (f_n, node))

    while len(frontier) > 0:
      node = heappop(frontier)[1]

      if highest_amt_stored_nodes < len(frontier):
            highest_amt_stored_nodes = len(frontier)

      if problem.goal_test(node.state):
        print("Nodes still left: ", len(frontier), "explored: ", len(explored), "most simultaneously stored nodes: ", highest_amt_stored_nodes)
        return node
      
      explored.add(node.state)
      for childNode in node.expand(problem):
            if childNode.state not in explored and not childNode.state in (x[1].state for x in frontier):
                distance = heuristic(problem.goal, childNode.state) + childNode.path_cost
                heappush(frontier, (distance, childNode))         
    return None

def ida_star_search(problem, heuristic):
    # Note: this algorithm is not needed to pass the testat
    raise NotImplementedError

### Test your algorithms with the Romanian text book example or with the SBB map

1. Romania

In [14]:
from math import sqrt, pow

romania_graph = UndirectedGraph(dict(
    Arad=dict(Zerind=75, Sibiu=140, Timisoara=118),
    Bucharest=dict(Urziceni=85, Pitesti=101, Giurgiu=90, Fagaras=211),
    Craiova=dict(Drobeta=120, Rimnicu=146, Pitesti=138),
    Drobeta=dict(Mehadia=75),
    Eforie=dict(Hirsova=86),
    Fagaras=dict(Sibiu=99),
    Hirsova=dict(Urziceni=98),
    Iasi=dict(Vaslui=92, Neamt=87),
    Lugoj=dict(Timisoara=111, Mehadia=70),
    Oradea=dict(Zerind=71, Sibiu=151),
    Pitesti=dict(Rimnicu=97),
    Rimnicu=dict(Sibiu=80),
    Urziceni=dict(Vaslui=142)))

romania_graph.locations = dict(
    Arad=(91, 492), Bucharest=(400, 327), Craiova=(253, 288),
    Drobeta=(165, 299), Eforie=(562, 293), Fagaras=(305, 449),
    Giurgiu=(375, 270), Hirsova=(534, 350), Iasi=(473, 506),
    Lugoj=(165, 379), Mehadia=(168, 339), Neamt=(406, 537),
    Oradea=(131, 571), Pitesti=(320, 368), Rimnicu=(233, 410),
    Sibiu=(207, 457), Timisoara=(94, 410), Urziceni=(456, 350),
    Vaslui=(509, 444), Zerind=(108, 531))

start = 'Arad'
goal = 'Bucharest'
problem = GraphProblem(start, goal, romania_graph)

# Define a heuristic function
# this calculates the aerial distance between two cities
def heuristic(a, b):
    x1, y1 = romania_graph.locations[a]
    x2, y2 = romania_graph.locations[b]
    return sqrt( pow(x1-x2, 2) + pow(y1-y2, 2))


To evaluate the goal node, try this function:

In [15]:
def evaluate(node):
    if node:
        print("The search algorithm reached " + node.state + " with a cost of " + str(node.path_cost) + ".")
        print("The actions that led to the solutions are the following: ")
        print(node.get_solution())
    else: 
        print('no solution found')


2. SBB

In [16]:
import json
import operator
import math


class TrainLine:
    def __init__(self, id, name):
        self.id = id
        self.name = name
        self.hubs = []
        self.hub_location = dict()

    def add_hub_at_km(self, hub, km):
        self.hubs.append(hub)
        self.hub_location[hub.name] = km

    def get_sorted_hubs(self):
        return sorted(self.hub_location.items(), key=operator.itemgetter(1))


class Hub:

    def __init__(self, name="", x=0, y=0):
        self.name = name
        self.x = x
        self.y = y

    def get_coordinates(self):
        return self.x, self.y


class SBB:
    def __init__(self):
        self.hubs = dict()
        self._train_lines = dict()

    def import_data(self, json_file_name):
        with open(json_file_name) as f:
            lines = json.load(f)
        for j in lines:
            if 'fields' not in j:
                continue
            train_line_id = j['fields']['linie']
            if train_line_id not in self._train_lines:
                train_line_name = j['fields']['linienname']
                self._train_lines[train_line_id] = TrainLine(train_line_id, train_line_name)
            hub = Hub()
            hub.name = treat_string(j['fields']['bezeichnung_bpk'])
            hub.x = j['fields']['geopos'][0]
            hub.y = j['fields']['geopos'][1]
            km = j['fields']['km']

            self._train_lines[train_line_id].add_hub_at_km(hub, km)
            self.hubs[hub.name] = hub

        print('successfully imported ' + str(len(self.hubs)) + ' hubs')
        print('successfully imported ' + str(len(self._train_lines)) + ' train lines')

    def create_map(self):
        map = dict()
        for line in self._train_lines:
            previous_hub_name = ""
            previous_km = -1
            for h in self._train_lines[line].get_sorted_hubs():
                hub_name = h[0]
                km = h[1]
                if previous_hub_name:
                    distance = abs(km - previous_km)
                    map.setdefault(hub_name, dict())
                    map.setdefault(previous_hub_name, dict())
                    map[hub_name].setdefault(previous_hub_name)
                    map[previous_hub_name].setdefault(hub_name)
                    map[hub_name][previous_hub_name] = distance
                    map[previous_hub_name][hub_name] = distance
                previous_hub_name = hub_name
                previous_km = km
        return map

    def get_hub_locations(self):
        locations = dict()
        for h in self.hubs:
            locations[h] = self.hubs[h].get_coordinates()
        return locations

    def get_distance_between(self, h1, h2):
        return math.sqrt((self.hubs[h1].x - self.hubs[h2].x) ** 2 + (self.hubs[h1].y - self.hubs[h2].y) ** 2) * 100


def treat_string(name):
    name = name.replace(" ", "_")
    name = name.replace('(', "")
    return name.replace(')', "")


In [18]:
sbb = SBB()
sbb.import_data('content/search/linie-mit-betriebspunkten.json')

start = 'Rotkreuz'
goal = 'Zermatt'
sbb_map = UndirectedGraph(sbb.create_map())
problem = GraphProblem(start, goal, sbb_map)

heuristic = sbb.get_distance_between

successfully imported 2787 hubs
successfully imported 401 train lines


To visaulize the map and the solution, use the following functions:

In [19]:
import folium

map_ch = folium.Map(location=[46.8, 8.33],
                    zoom_start=8, tiles="Stamen Toner")

for hub in sbb.hubs:
    folium.CircleMarker(location=[sbb.hubs[hub].x, sbb.hubs[hub].y],
                        radius=2,
                        weight=4).add_to(map_ch)
map_ch


ValueError: Custom tiles must have an attribution.

In [20]:
def show_solution(map, goal_node):
      
    points = []
    
    for hub in goal_node.get_path_from_root():
        points.append([sbb.hubs[hub.state].x, sbb.hubs[hub.state].y])
        folium.CircleMarker(location=[sbb.hubs[hub.state].x, sbb.hubs[hub.state].y], color='red',
                        radius=2,
                        weight=4).add_to(map)
    folium.PolyLine(points, color='red').add_to(map)
    return map




In [22]:
import itertools

goal_node = greedy_search(problem, heuristic)
print(goal_node.path_cost)

Nodes still left:  13 explored:  2217 most simultaneously stored nodes:  28
359.0819999999998


In [23]:
show_solution(map_ch, goal_node)

NameError: name 'map_ch' is not defined

In [44]:
goal_node = a_star_search(problem, heuristic)
print(goal_node.path_cost)
# show_solution(map_ch, goal_node)

Nodes still left:  33 explored:  2021 most simultaneously stored nodes:  42
330.9589999999999
